In [ ]:
import sympy
import IPython.display as disp

In [ ]:
import helper_functions as hf

In [ ]:
sympy.init_printing()

dt = sympy.Symbol("\Delta t", real=True)
g = sympy.Symbol("g", real=True)

**Integrated gyroscope measurements**

In [ ]:
delta_angle_x = sympy.Symbol("\Delta \Omega_x", real=True)
delta_angle_y = sympy.Symbol("\Delta \Omega_y", real=True)
delta_angle_z = sympy.Symbol("\Delta \Omega_z", real=True)
delta_angle = sympy.Matrix([delta_angle_x, delta_angle_y, delta_angle_z])
delta_angle

**Integrated accelerometer measurements**

In [ ]:
delta_velocity_x = sympy.Symbol("\Delta v_x", real=True)
delta_velocity_y = sympy.Symbol("\Delta v_y", real=True)
delta_velocity_z = sympy.Symbol("\Delta v_z", real=True)
delta_velocity = sympy.Matrix([delta_velocity_x, delta_velocity_y, delta_velocity_z])
delta_velocity

**Combined IMU measurements**

In [ ]:
u = sympy.Matrix([delta_angle, delta_velocity])
u

**Gyroscope noise variance**

In [ ]:
delta_angle_x_var = sympy.Symbol("\sigma^2_{\Delta \Omega_x}", real=True)
delta_angle_y_var = sympy.Symbol("\sigma^2_{\Delta \Omega_y}", real=True)
delta_angle_z_var = sympy.Symbol("\sigma^2_{\Delta \Omega_z}", real=True)
delta_angle_var = sympy.Matrix([delta_angle_x_var, delta_angle_y_var, delta_angle_z_var])
delta_angle_var

**Accelerometer noise variance**

In [ ]:
delta_velocity_x_var = sympy.Symbol("\sigma^2_{\Delta v_x}", real=True)
delta_velocity_y_var = sympy.Symbol("\sigma^2_{\Delta v_y}", real=True)
delta_velocity_z_var = sympy.Symbol("\sigma^2_{\Delta v_z}", real=True)
delta_velocity_var = sympy.Matrix([delta_velocity_x_var, delta_velocity_y_var, delta_velocity_z_var])
delta_velocity_var

**Combined IMU noise variance**

In [ ]:
u_var = sympy.Matrix.diag(delta_angle_x_var, delta_angle_y_var, delta_angle_z_var, delta_velocity_x_var, delta_velocity_y_var, delta_velocity_z_var)
u_var

## Definition of state vector
### orientation
Using w-x-y-z convention

In [ ]:
qw = sympy.Symbol("q_w", real=True)
qx = sympy.Symbol("q_x", real=True)
qy = sympy.Symbol("q_y", real=True)
qz = sympy.Symbol("q_z", real=True)
q = sympy.Matrix([qw, qx, qy, qz])
q

In [ ]:
R_to_earth = hf.quat2Rot(q)
R_to_body = R_to_earth.T
R_to_earth

### velocity
Using the map (ENU) reference frame, following the ROS convention.

In [ ]:
vx = sympy.Symbol("v^{ENU}_x", real=True)
vy = sympy.Symbol("v^{ENU}_y", real=True)
vz = sympy.Symbol("v^{ENU}_z", real=True)
v = sympy.Matrix([vx, vy, vz])
v

### Position
Using the map (ENU) reference frame, following the ROS convention.

In [ ]:
px = sympy.Symbol("p^{ENU}_x", real=True)
py = sympy.Symbol("p^{ENU}_y", real=True)
pz = sympy.Symbol("p^{ENU}_z", real=True)
p = sympy.Matrix([px, py, pz])
p

### Delta angle bias

In [ ]:
delta_angle_bias_x = sympy.Symbol("\Delta \Omega_{bias,x}", real=True)
delta_angle_bias_y = sympy.Symbol("\Delta \Omega_{bias,y}", real=True)
delta_angle_bias_z = sympy.Symbol("\Delta \Omega_{bias,z}", real=True)
delta_angle_bias = sympy.Matrix([delta_angle_bias_x, delta_angle_bias_y, delta_angle_bias_z])
delta_angle_true = delta_angle - delta_angle_bias
delta_angle_true

### Delta velocity bias

In [ ]:
delta_velocity_bias_x = sympy.Symbol("\Delta v_{bias, x}", real=True)
delta_velocity_bias_y = sympy.Symbol("\Delta v_{bias, y}", real=True)
delta_velocity_bias_z = sympy.Symbol("\Delta v_{bias, z}", real=True)
delta_velocity_bias = sympy.Matrix([delta_velocity_bias_x, delta_velocity_bias_y, delta_velocity_bias_z])
delta_velocity_true = delta_velocity - delta_velocity_bias
delta_velocity_true

### State vector

In [ ]:
state = sympy.Matrix([q, v, p, delta_angle_bias, delta_velocity_bias])
state

## State propagation/prediction
### Orientation

In [ ]:
q_new = hf.quat_mult(q, sympy.Matrix([1, 0.5 * delta_angle_true[0], 0.5 * delta_angle_true[1], 0.5 * delta_angle_true[2]]))
q_new

### Velocity
subtract gravity vector (not sure about sign)

In [ ]:
v_new = v + R_to_earth * delta_velocity_true + sympy.Matrix([0, 0, -g]) * dt
v_new

### Position

In [ ]:
p_new = p + v * dt
p_new

### IMU bias

In [ ]:
delta_angle_bias_new = delta_angle_bias
delta_velocity_bias_new = delta_velocity_bias

### State

In [ ]:
state_new = sympy.Matrix([q_new, v_new, p_new, delta_angle_bias_new, delta_velocity_bias_new])
state_new

## State propagation/prediction jacobians

In [ ]:
F = state_new.jacobian(state)
F

### Process Noise

In [ ]:
G = state_new.jacobian(u)
G

In [ ]:
Q = G * u_var * G.T
Q

In [ ]:
P = hf.create_symmetric_cov_matrix([sympy.shape(state)[0], sympy.shape(state)[0]])
# P

In [ ]:
tmp1 = F * P

In [ ]:
tmp2 = tmp1 * F.T

In [ ]:
P_new = tmp2 + Q
for row in range(sympy.shape(P_new)[0]):
    for col in range(sympy.shape(P_new)[1]):
        if row > col:
            P_new[row, col] = 0

In [ ]:
P_new_simple = sympy.cse(P_new, sympy.utilities.iterables.numbered_symbols(prefix='tmp'), optimizations='basic')

In [ ]:
with open("test.c", "w") as f:
    hf.write_subexpressions(f, P_new_simple[0])
    hf.write_matrix(f, sympy.Matrix(P_new_simple[1]), "P_new", True)
